Aşağıdaki not defterlerini çalıştırmak için, henüz yapmadıysanız, .env dosyasının içine `OPENAI_API_KEY` olarak openai anahtarını ayarlamanız gerekir.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Sonraki adımda, Embedding İndeksini bir Pandas Dataframe'e yükleyeceğiz. Embedding İndeksi, `embedding_index_3m.json` adlı bir JSON dosyasında saklanmaktadır. Embedding İndeksi, Ekim 2023 sonuna kadar olan her YouTube transkripti için Embedding'leri içermektedir.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Sonraki adımda, sorgu için Embedding İndeksinde arama yapacak `get_videos` adlı bir fonksiyon oluşturacağız. Fonksiyon, sorguya en benzer ilk 5 videoyu döndürecektir. Fonksiyon şu şekilde çalışır:

1. Öncelikle, Embedding İndeksinin bir kopyası oluşturulur.
2. Daha sonra, OpenAI Embedding API kullanılarak sorgunun Embedding'i hesaplanır.
3. Ardından, Embedding İndeksinde `similarity` adlı yeni bir sütun oluşturulur. `similarity` sütunu, sorgu Embedding'i ile her video segmentinin Embedding'i arasındaki kosinüs benzerliğini içerir.
4. Sonra, Embedding İndeksi `similarity` sütununa göre filtrelenir. Embedding İndeksi, kosinüs benzerliği 0.75 veya daha yüksek olan videoları içerecek şekilde filtrelenir.
5. Son olarak, Embedding İndeksi `similarity` sütununa göre sıralanır ve en benzer ilk 5 video döndürülür.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Bu fonksiyon çok basittir, sadece arama sorgusunun sonuçlarını yazdırır.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. İlk olarak, Gömülü Dizin bir Pandas Dataframe'e yüklenir.
2. Sonra, kullanıcıdan bir sorgu girmesi istenir.
3. Ardından, sorgu için Gömülü Dizin'de arama yapmak üzere `get_videos` fonksiyonu çağrılır.
4. Son olarak, sonuçları kullanıcıya göstermek için `display_results` fonksiyonu çağrılır.
5. Daha sonra kullanıcıdan başka bir sorgu girmesi istenir. Bu işlem kullanıcı `exit` girene kadar devam eder.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc.tr.png)

Bir sorgu girmeniz istenecek. Bir sorgu girin ve enter tuşuna basın. Uygulama, sorguyla ilgili videoların bir listesini döndürecektir. Uygulama ayrıca sorunun cevabının bulunduğu videodaki yere bir bağlantı da döndürecektir.

Denemek için bazı sorgular şunlardır:

- Azure Machine Learning nedir?
- Konvolüsyonel sinir ağları nasıl çalışır?
- Sinir ağı nedir?
- Azure Machine Learning ile Jupyter Notebooks kullanabilir miyim?
- ONNX nedir?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Feragatname**:  
Bu belge, AI çeviri servisi [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba gösterilse de, otomatik çevirilerin hatalar veya yanlışlıklar içerebileceğini lütfen unutmayınız. Orijinal belge, kendi dilinde yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımı sonucu oluşabilecek yanlış anlamalar veya yorum hatalarından sorumlu değiliz.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
